<a href="https://colab.research.google.com/github/mildinvestor/katago-colab/blob/master/build_colab_katago_gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Build colab-katago-gd client binaries

In [ ]:
GIT_REPOSITORY_URL = 'https://github.com/mildinvestor/katago-colab.git'

# Install Golang and git
!add-apt-repository ppa:longsleep/golang-backports -y 1>/dev/nul
!apt-get update 1>/dev/nul
!apt-get install --yes git golang-go 1>/dev/nul
%env GOPATH=/root/go

# Display version
!git version
!go version

%cd /content

# Clone katago-colab
!rm -rf katago-colab
!git clone $GIT_REPOSITORY_URL

# Build colab-katago
%cd katago-colab
!chmod +x ./package.sh
!./package.sh

# File list
!ls -l ./bin/

# Zip and donwload
!zip -r ./colab-katag-gd-bin.zip ./bin/*.zip
from google.colab import files
files.download("./colab-katag-gd-bin.zip")

#Compile KataGo OPENCL for colab-katago

https://github.com/lightvector/KataGo/blob/master/Compiling.md

In [ ]:
#@markdown KataGo version
TAG = "v1.9.1" #@param {type:"string"}

# Install libraries
!apt-get update 1>/dev/nul
!apt-get install --yes git libzip-dev cmake 1>/dev/nul

import subprocess
gpu_name = str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
print("GPU: {}".format(gpu_name))

%cd /content

# Checkout source of KataGo
!git clone https://github.com/lightvector/KataGo.git
!git checkout $TAG

# Compile
%cd KataGo/cpp
!cmake . -DUSE_BACKEND=OPENCL
#!cmake . -DUSE_BACKEND=OPENCL -DUSE_TCMALLOC=1
!make

# Display version
!./katago version

# Download KataGo binary
!zip -r ./katago.zip ./katago
from google.colab import files
files.download("./katago.zip")

#Create a tuning file of KataGo

In [ ]:
WEIGHT_URL = 'https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s9854456576-d2405111631.bin.gz'
!wget --quiet $WEIGHT_URL -O /content/weight.bin.gz

%cd /content
!./KataGo/cpp/katago version
# New tuning features in KataGo v1.9.0 and later
!./KataGo/cpp/katago tuner -model ./weight.bin.gz -config ./KataGo/cpp/configs/gtp_example.cfg
#!./KataGo/cpp/katago benchmark -model ./weight.bin.gz -config ./KataGo/cpp/configs/gtp_example.cfg -v 300

!zip -r ./opencltuning.zip /root/.katago/opencltuning
from google.colab import files
files.download("./opencltuning.zip")

#The following sections are for debugging only, you can ignore.

##GPU information

In [ ]:
!nvidia-smi